In [1]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array      # New import
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from ipywidgets import FileUpload, Image, VBox
from IPython.display import display

dataset_path = "PokemonData"
img_width, img_height = 150, 150
batch_size = 32

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,       
    width_shift_range=0.2,   
    height_shift_range=0.2,  
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,  

    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(  

    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.


In [8]:
# ---- Model Architecture ----
model = Sequential([
    Conv2D(32, (3, 3), activation=LeakyReLU(alpha=0.1), input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation=LeakyReLU(alpha=0.1)), # Increased number of filters
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation=LeakyReLU(alpha=0.1)),  # Added another dense layer
    BatchNormalization(),
    Dropout(0.5),             
    Dense(150, activation='softmax') 
])

# ---- Model Compilation and Training ----
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)  

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  



In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,             
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,  

    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/30
172/172 [==============================] - 152s 873ms/step - loss: 5.0150 - accuracy: 0.0469 - val_loss: 5.8877 - val_accuracy: 0.0156 - lr: 0.0010
Epoch 2/30
172/172 [==============================] - 109s 632ms/step - loss: 4.0568 - accuracy: 0.1007 - val_loss: 5.0521 - val_accuracy: 0.0383 - lr: 0.0010
Epoch 3/30
172/172 [==============================] - 109s 634ms/step - loss: 3.5916 - accuracy: 0.1568 - val_loss: 3.2335 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 4/30
172/172 [==============================] - 109s 630ms/step - loss: 3.2027 - accuracy: 0.2134 - val_loss: 3.0113 - val_accuracy: 0.2516 - lr: 0.0010
Epoch 5/30
172/172 [==============================] - 109s 631ms/step - loss: 2.9842 - accuracy: 0.2555 - val_loss: 2.6760 - val_accuracy: 0.3164 - lr: 0.0010
Epoch 6/30
172/172 [==============================] - 110s 640ms/step - loss: 2.7478 - accuracy: 0.2957 - val_loss: 2.7806 - val_accuracy: 0.2977 - lr: 0.0010
Epoch 7/30
172/172 [==========================

In [10]:
model.save("pokemon_cnn_model_improved.h5")

c:\Users\Vincent\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from ipywidgets import FileUpload, Image, VBox
from IPython.display import display

# Load the saved model
model = load_model("pokemon_cnn_model_improved.h5")

# Path to the directory containing the Pokémon images
dataset_path = "PokemonData"

# Define image dimensions
img_width, img_height = 150, 150

# Function to predict Pokémon from an image file and return the predicted class along with its image
@tf.function(reduce_retracing=True) 
def predict_pokemon_with_image(img_array):
    img_array = tf.image.resize(img_array, [img_width, img_height])
    img_array = tf.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  
    return img_array
    
# Function to handle the uploaded image and display predictions
def on_upload_change(change):
    uploaded_file = change['new']
    if uploaded_file:
        content = uploaded_file[0]['content']
        img = load_img(io.BytesIO(content))
        img_array = img_to_array(img)

        # Predict and display the results
        img_array = predict_pokemon_with_image(img_array)
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)
        
        # Get the label (Pokémon name) and similarity percentage of the predicted class
        predicted_pokemon = train_generator.class_indices
        predicted_pokemon = dict((v,k) for k,v in predicted_pokemon.items())
        predicted_pokemon = predicted_pokemon[predicted_class]
        similarity_percentage = round(np.max(prediction[0]) * 100, 2)  # Get percentage and round to 2 decimal places

        # Load the image of the predicted Pokémon class
        predicted_image_path = os.path.join(dataset_path, predicted_pokemon, os.listdir(os.path.join(dataset_path, predicted_pokemon))[0])
        predicted_img = load_img(predicted_image_path, target_size=(img_width, img_height))

        # Plot the images
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(img)
        axes[0].set_title("Uploaded Image")
        axes[0].axis('off')
        axes[1].imshow(predicted_img)
        axes[1].set_title(f"Predicted Pokémon: {predicted_pokemon} ({similarity_percentage}%)") # Add similarity percentage
        axes[1].axis('off')
        plt.show()


upload_button = FileUpload(accept='image/*', multiple=False)
upload_button.observe(on_upload_change, names='value')


display(VBox([upload_button]))
